# 1. Introduction to rgeoda

## Install rgeoda for R

rgeoda is a R package that wraps all core functions of spatial data analysis in GeoDa and libgeoda. Unlike the desktop software GeoDa, libgeoda is a non-UI and feature focused C++ library that is designed for programmers to do spatial data analysis using their favoriate programming languages (R, Python, Java etc.). It also aims to be easily integratd with other libraries, softwares or systems on different platforms.

For testing stage, the easiest way to install the development version of rgeoda is using rgeoda's source package:
```R
install.packages("https://github.com/lixun910/rgeoda/archive/0.0.1.tar.gz")
```

For windows R users, please use the following source package:
```R
install.packages("https://github.com/lixun910/rgeoda/releases/download/0.0.1/rgeoda_0.0.1.zip")
```

If everything installed without error, you should be able to load rgeoda:

In [1]:
library(rgeoda)

## Load Geospatial Data

In this note, we will use an ESRI Shapefile natregimes.shp comes with the package:

In [2]:
nat_path <- system.file("extdata", "natregimes.shp", package = "rgeoda")
nat_path

[1] "/Library/Frameworks/R.framework/Versions/3.5/Resources/library/rgeoda/extdata/natregimes.shp"

Use the path above (or the path to your own dataset), we can create a GeoDa instance, and the GeoDa instance is the main entry point of rgeoda.

In [3]:
gda <- GeoDa(nat_path)

rgeoda provides some functions to check the meta data of the loaded dataset:


In [4]:
num_obs <- gda$GetNumObs()
num_cols <- gda$GetNumCols()
field_types <- gda$GetFieldTypes()
field_names <- gda$GetFieldNames()

num_obs
num_cols

[1] 3085

[1] 73

## Access Table Data

One can call function `GetNumericCol(string col_name)` to get numeric data (vector type) from GeoDa instance. For example, to get data of column “HR60”:

In [5]:
hr60 <- gda$GetNumericCol("HR60")
hr60[1:20]

[1]  0.000000  0.000000  1.863863  2.612330  0.000000  0.000000  7.976390
 [8]  1.011173 11.529039  0.000000  0.000000  3.574045  0.000000  0.000000
[15]  0.000000  0.000000  2.945942  0.000000  0.000000  0.000000

## Spatial Weights

One can call function `CreateContiguityWeights(string poly_id="", bool is_queen=true, int order=1, bool include_lower_order=false)` to create a Queen or Rook contiguity weights:

In [6]:
queen_w <- gda$CreateContiguityWeights(is_queen=TRUE)

To access the properties of the created weights object, one can just access the attributes of the returned weights object queen_w:

In [7]:
cat("weight_type: ", queen_w$weight_type, 
    "\nis_symmetric: ", queen_w$is_symmetric, 
    "\nsparsity:",queen_w$sparsity,
    "\ndensity:",queen_w$density, 
    "\nmin_nbrs:",queen_w$min_nbrs, 
    "\nmax_nbrs:",queen_w$max_nbrs,
    "\nmean_nbrs:",queen_w$mean_nbrs, 
    "\nmedian_nbrs:",queen_w$median_nbrs, "\n")

weight_type:  gal_type 
is_symmetric:  TRUE 
sparsity: 0 
density: 0.190896 
min_nbrs: 1 
max_nbrs: 14 
mean_nbrs: 5.889141 
median_nbrs: 6 


## Spatial Data Analysis

### Local Spatial Autocorrelation

Using the created Queen weights queen_w and the data hr60, we can call function `LISA(GeoDaWeight w, vector data)` to compute the local spatial autocorrelation of variable “HR60”.

In [8]:
lisa <- gda$LISA(queen_w, hr60)

We can access the LISA results by calling the “getter” methods from the returned LISA object:

In [9]:
lags <- lisa$GetLagValues()
lags[1:20]

[1] -0.6293091 -0.3530115 -0.5508477 -0.4217541 -0.5088273 -0.2004406
 [7] -0.4434569  0.4960453  0.2903570  0.9109106  0.3326043 -0.2865061
[13] -0.5457504 -0.7972207 -0.5379562 -0.5026332 -0.6643260 -0.6234103
[19] -0.6721546 -0.7972207

Get local moran values:

In [10]:
lms <- lisa$GetLocalMoranValues()
lms[1:20]

[1]  0.50169822  0.28142804  0.25742058  0.14121956  0.40564765  0.15979543
 [7] -0.27254825 -0.30667660  0.36103503 -0.72619680 -0.26515906  0.04716326
[13]  0.43508356  0.63556087  0.42886986  0.40070958  0.18321405  0.49699558
[19]  0.53585554  0.63556087

Get pseudo-p values:

In [11]:
pvals <- lisa$GetLocalSignificanceValues()
pvals[1:20]

[1] 0.090 0.302 0.099 0.088 0.120 0.397 0.187 0.073 0.264 0.068 0.216 0.338
[13] 0.115 0.029 0.062 0.114 0.037 0.096 0.027 0.013

Get LISA category values:

In [12]:
cats <- lisa$GetClusterIndicators()
cats[1:20]

[1] 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 2 0 2 2

You can easily re-run the LISA computation by calling its Run() function. For example, re-run the above LISA exampe using 9999 permutations:

In [13]:
lisa$SetNumPermutations(9999)
lisa$Run()

NULL

NULL

Display the p-values after 9999 permutations:

In [14]:
pvals <- lisa$GetLocalSignificanceValues()
pvals[1:20]

[1] 0.0874 0.2911 0.0942 0.1024 0.1149 0.4212 0.1698 0.0735 0.2430 0.0660
[11] 0.2079 0.3471 0.1012 0.0250 0.0739 0.1256 0.0353 0.0902 0.0180 0.0075

Since rgeoda is using GeoDa’s C++ code, by default, rgeoda uses multi-threading to accelerate the computation of Local Moran. One can also specify how many threads to run the computation:

In [15]:
lisa$SetNumThreads(4)
lisa$Run()

NULL

NULL

Display the p-values after re-run LISA with 4 threads:

In [16]:
pvals <- lisa$GetLocalSignificanceValues()
pvals[1:20]

[1] 0.0874 0.2911 0.0942 0.1024 0.1149 0.4212 0.1698 0.0735 0.2430 0.0660
[11] 0.2079 0.3471 0.1012 0.0250 0.0739 0.1256 0.0353 0.0902 0.0180 0.0075

## Clustering

### SKATER

Using the function `SKATER(int k, GeoDaWeight w, Vector column_names, String distance_method='euclidean')` to run a spatially constrained clustering on current dataset.

In [17]:
skater <- gda$SKATER(10, queen_w, c("HR60", "PO60"))
skater

[1]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  [25]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  [49]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  [73]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  [97]  1  4  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 [121]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 [145]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  4  4  1  1  1  1  1  1  1
 [169]  4  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 [193]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 [217]  1  1  1  1  1  1  1  1  1  1  1  1  4  4  1  1  1  1  1  1  1  1  1  1
 [241]  1  1  1  1  1  1  1  4  1  1  4  1  1  1  1  1  1  1  1  1  1  1  1  1
 [265]  1  1  1  1  1  1  1  1  1  1  4  4  4  4  4  1  1  1  1  1  4  1  1  1
 [289]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  4  1  1  4  4  1  1  1  4  4
 [313]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  4  1  1  1  1  1  1
 [337]  1  4  4  1  1  1  4  4  1  4  1  4  4  4  1  1  1  1  1  4  1  1  1  1
 [361]  1  1  1  1  1  1  1  1  1  1  1  1  4  1  1  1  4  1  4  1  1  1  1  1
 [385]  4  1  1  1  1  4  1  1  1  1  1  1  1  1  4  1  1  1  1  1  1  1  1  1
 [409]  1  1  1  1  4  4  1  4  1  4  1  1  1  1  1  1  1  1  4  4  4  1  1  1
 [433]  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  4  1  1
 [457]  4  1  4  4  1  4  1  1  1  1  1  1  1  1  1  4  1  1  1  4  1  1  1  1
 [481]  1  1  1  4  1  1  1  1  1  1  1  1  1  1  4  4  1  1  1  1  1  1  1  1
 [505]  1  1  4  1  4  1  1  1  4  1  1  4  1  1  1  4  4  4  4  1  1  1  1  1
 [529]  1  1  1  1  1  1  1  1  1  4  1  1  1  4  1  4  1  4  4  1  1  1  1  1
 [553]  1  1  1  4  1  1  1  1  1  1  4  4  1  1  1  1  1  1  1  4  1  4  1  1
 [577]  1  1  1  1  1  1  1  4  4  1  4  4  1  4  4  4  4  1  1  1  1  1  1  1
 [601]  4  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  4  4  1  1
 [625]  1  1  1  1  1  1  1  2  2  2  2  2  2  2  4  2  1  1  4  4  1  4  4  4
 [649]  4  4  2  4  2  2  2  2  2  2  2  4  4  1  4  1  1  1  2  2  4  1  2  1
 [673]  1  1  1  1  1  1  1  1  2  2  2  2  2  2  2  1  1  1  2  2  2  2  1  2
 [697]  1  1  4  1  1  1  1  1  1  1  1  2  1  1  4  4  2  2  1  4  2  4  2  2
 [721]  2  2  2  2  1  2  2  4  2  2  1  2  1  4  1  1  1  4  2  2  2  2  4  2
 [745]  1  2  1  1  1  1  1  1  1  1  1  1  1  1  2  4  4  2  2  4  2  4  1  4
 [769]  1  1  1  1  2  2  1  1  1  1  1  1  2  2  2  2  1  1  1  1  1  1  1  1
 [793]  1  2  1  1  2  2  2  2  2  1  4  1  4  2  2  2  2  2  2  1  2  2  1  1
 [817]  1  1  5  1  1  1  1  4  1  4  1  2  2  1  4  1  2  1  4  4  5  2  4  2
 [841]  1  2  2  1  2  2  4  2  2  1  2  2  1  1  1  1  4  1  1  1  1  2  1  1
 [865]  1  4  2  2  2  2  4  2  1  1  4  2  4  1  1  1  4  2  2  1  1  2  1  1
 [889]  1  2  1  2  1  2  1  4  2  1  1  1  2  2  2  2  2  1  2  1  4  1  2  4
 [913]  1  2  1  1  2  4  2  2  2  2  2  1  1  2  2  1  1  1  1  1  1  1  1  1
 [937]  1  1  2  2  4  2  4  1  2  2  2  2  1  2  2  1  1  1  1  1  1  4  2  4
 [961]  2  1  2  1  2  2  2  4  2  1  2  4  2  2  1  1  2  2  4  1  1  1  2  1
 [985]  1  2  4  4  2  1  1  2  1  2  2  2  1  1  1  1  1  1  1  1  1  1  1  1
[1009]  1  1  1  1  1  4  2  1  2  1  2  1  2  2  2  2  1  2  2  2  1  1  1  2
[1033]  2  4  1  2  2  2  1  1  4  2  1  1  1  2  1  1  2  1  2  2  1  4  1  2
[1057]  1  1  2  1  4  2  2  4  2  4  2  2  4  4  2  2  2  1  2  1  2  2  1  1
[1081]  2  2  1  1  1  2  2  2  2  2  1  1  2  1  2  1  2  2  1  2  2  1  2  1
[1105]  1  1  1  1  4  1  1  1  1  1  1  1  1  1  1  2  1  2  1  2  2  1  1  1
[1129]  2  1  2  2  2  2  2  2  2  1  2  2  2  1  1  1  2  1  1  2  2  4  2  1
[1153]  2  4  4  4  1  1  4  1  1  2  2  2  2  4  2  2  1  2  1  2  1  2  1  4
[1177]  2  2  2  1  2  2  1  1  2  2  1  4  1  2  1  1  2  1  1  1  2  2  2  1
[1201]  2  2  2  2  2  2  2  1  1  2  4  1  1  1  1  